# Initialize

In [ ]:
# Options

## Debugging level
debug_level = 0


In [ ]:
# Initialize framework
tools = init_preprocessing(debug_level)
%matplotlib inline


In [ ]:
# Read anonymized data
atlas           = pd.read_pickle(constants.PREPROCESSED_ATLAS_FILE_PATH)
forecast        = pd.read_pickle(constants.PREPROCESSED_FORECAST_DATA_FILE_PATH)
revenue2020     = pd.read_pickle(constants.PREPROCESSED_REVENUE2020_FILE_PATH)
revenue2020A    = pd.read_pickle(constants.PREPROCESSED_REVENUE2020A_FILE_PATH)
atlas2          = pd.read_pickle(constants.PREPROCESSED_ATLAS_2_FILE_PATH)


In [ ]:
# Build lookup tables from the data
lut_address_state = forecast["Mailing State/Province"].drop_duplicates().to_dict()
lut_record_type = forecast["Organization Record Type"].drop_duplicates().to_dict()
lut_customer_type = forecast["ATLAS Customer Type"].drop_duplicates().to_dict()
lut_vendor_id = revenue2020A["Vendor ID"].drop_duplicates().to_dict()
lut_address = atlas2["Address"].drop_duplicates().to_dict()
lut_address_lat = atlas2["Lat"].drop_duplicates().to_dict()
lut_address_lon = atlas2["Long"].drop_duplicates().to_dict()

# Write lookup tables to file
tool_lookup_tables.write_lookup_table(lut_address_state, constants.LUT_ADDRESS_STATE_FILE_PATH)
tool_lookup_tables.write_lookup_table(lut_record_type, constants.LUT_RECORD_TYPE_FILE_PATH)
tool_lookup_tables.write_lookup_table(lut_customer_type, constants.LUT_CUSTOMER_TYPE_FILE_PATH)
tool_lookup_tables.write_lookup_table(lut_vendor_id, constants.LUT_VENDOR_ID_FILE_PATH)
tool_lookup_tables.write_lookup_table(lut_address, constants.LUT_ADDRESS_FILE_PATH)
tool_lookup_tables.write_lookup_table(lut_address_lat, constants.LUT_ADDRESS_LAT_FILE_PATH)
tool_lookup_tables.write_lookup_table(lut_address_lon, constants.LUT_ADDRESS_LON_FILE_PATH)


In [ ]:
# tool_lookup_tables.read_lookup_table( constants.LUT_ADDRESS_STATE_FILE_PATH)
# tool_lookup_tables.read_lookup_table( constants.LUT_RECORD_TYPE_FILE_PATH)
# tool_lookup_tables.read_lookup_table( constants.LUT_CUSTOMER_TYPE_FILE_PATH)
# tool_lookup_tables.read_lookup_table( constants.LUT_VENDOR_ID_FILE_PATH)
# tool_lookup_tables.read_lookup_table( constants.LUT_ADDRESS_FILE_PATH)
# tool_lookup_tables.read_lookup_table( constants.LUT_ADDRESS_LAT_FILE_PATH)
# tool_lookup_tables.read_lookup_table( constants.LUT_ADDRESS_LON_FILE_PATH)

In [ ]:
# Build combined data table

## Get list of customers to use as the index
customer_name_mapping = tool_lookup_tables.read_customer_name_mapping()
customer_list = list(customer_name_mapping.keys())



# combined_data = pd.DataFrame(index=)


# PREPROCESSED_COMBINED_DATA_FILE_PATH

In [ ]:


a=["name1", "name2", "name3", "name4"]

a={10, 32,150,47,2,1,8,4,71,,3,2,4,8720, 3, 4}


